In [15]:
import re
import pandas as pd
import numpy as np

In [56]:
guarant_tech = pd.read_csv('guarant_tech.csv')
guarant_user = pd.read_csv('guarant_user.csv')
undefined_users = pd.read_csv('out.csv')

In [57]:
undefined_users = undefined_users[undefined_users['target'] == 1]

In [59]:
guarant_tech['target'] = 0
guarant_user['target'] = 1
guarant_user.head()
data = pd.concat([guarant_tech, guarant_user, undefined_users], ignore_index=True)
print(data.shape)
data.head()

(164621, 2)


,url,target
0,w-74721.fp.kaspersky-labs.com,0
1,passport-authproxy.taxi.yandex.net,0
2,r8---sn-jvhnu5g-n8me.googlevideo.com,0
3,sonar-gmp1-1.xx.fbcdn.net,0
4,4wll4l5gec.execute-api.us-west-2.amazonaws.com,0


In [60]:
tech = data[data['target'] == 0].sample(frac=1).head(40000)
user = data[data['target'] == 1]
data = pd.concat([tech, user], ignore_index=True).sample(frac=1)

data['target'].value_counts()

0    40000
1    19598
Name: target, dtype: int64

In [20]:
regexp = re.compile(r'\d')

data['len'] = list(map(len, data['url'].values))
data['dots'] = list(map(lambda x: x.count('.'), data['url'].values))
data['hyphen'] = list(map(lambda x: x.count('-'), data['url'].values))
data['www'] = list(map(lambda x: int('www' in x), data['url'].values))
data['numbers'] =  list(map(lambda x: sum(c.isdigit() for c in x), data['url'].values))

data.head()

,url,target,len,dots,hyphen,www,numbers
27888,city.travel,1,11,1,0,0,0
25513,4txt.ru,1,7,1,0,0,1
38189,snapwidget.com,1,14,1,0,0,0
12051,cdn1.hype.tech,0,14,2,0,0,1
31093,turbina.ru,1,10,1,0,0,0


In [71]:
import matplotlib.pyplot as plt

words = []
url_words = map(lambda x: x.split('.')[:-1], data['url'])
url_words = [item for sublist in url_words for item in sublist]
url_words = pd.Series(url_words)
words_frequency = pd.DataFrame(url_words.value_counts(), columns=['count'])
words_frequency.head(10).reset_index().plot.barh(x='index', y='count')


<AxesSubplot:>

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = data['url'].values
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(corpus).toarray()
print(vectorizer.get_feature_names())

['01', '02', '03', '045e', '10', '100', '101', '104', '107', '11', '116', '117', '118', '119', '12', '123', '129', '13', '138_ts', '14', '140', '142', '15', '154', '157', '16', '163', '168', '17', '173', '178_ts', '18', '181', '185', '188', '19', '192', '194', '195', '1c', '20', '202', '203', '205', '21', '212', '213', '216', '217', '219_ts', '22', '224', '23', '233', '24', '240', '242', '244', '248', '25', '250', '252', '254', '26', '29', '2gis', '2mdn', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '41', '43', '44', '45', '49', '50', '51', '52', '54', '59', '5ka', '62', '64', '65', '6sekretov', '72', '74', '78', '85', '87', '88', '94', '95', '99', '_dns', '_udp', 'a20a7', 'aa', 'aamsqec', 'abs', 'ac', 'academic', 'academy', 'accesspoint', 'account', 'ad', 'addr', 'adidas', 'admin', 'adr', 'ads', 'adsco', 'aero', 'ahblatp', 'ai', 'aif', 'akadns', 'akama', 'akamai', 'akamaiedge', 'akamaihd', 'akamaized', 'alb', 'alibaba', 'alicdn', 'aliexpress', 'all', 'alpha', 'alternate

In [23]:
data = pd.concat([data, pd.DataFrame(X)], axis=1)
X = data.drop(['target'], axis=1).drop(['url'], axis=1)
y = data['target']

In [24]:
X.head()

,len,dots,hyphen,www,numbers,0,1,2,3,4,...,990,991,992,993,994,995,996,997,998,999
0,15,2,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,2,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,64,3,0,0,19,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26,3,0,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.naive_bayes import  MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, stratify=y, shuffle=True)

multi_clf = MultinomialNB()
multi_clf.fit(X_train, y_train)

MultinomialNB()

In [27]:
data.groupby('target').agg(['mean', 'std'])

len                 dots             hyphen                 www  \
             mean        std      mean       std     mean       std      mean   
target                                                                          
0       33.354100  19.363013  2.665050  2.106984  1.05410  1.712869  0.005100   
1       14.994744   4.592906  1.538626  0.508049  0.20946  0.456126  0.215328   

                   numbers            ...      995                 996  \
             std      mean       std  ...     mean       std      mean   
target                                ...                                
0       0.071234  8.853800  8.534487  ...  0.00025  0.015810  0.000250   
1       0.411060  0.259312  0.854352  ...  0.00051  0.022584  0.000204   

                       997                 998                 999            
             std      mean       std      mean       std      mean       std  
target                                                                        
0       0.015810  0.000250  0.015810  0.001000  0.031608  0.000350  0.018705  
1       0.014285  0.000153  0.012372  0.001378  0.037093  0.000051  0.007143  

[2 rows x 2010 columns]

In [28]:
print('Accuracy score', accuracy_score(y_test, multi_clf.predict(X_test)))
print('-----------------------------------------')
print('Confusion Matrix')
print(confusion_matrix(y_test, multi_clf.predict(X_test)))
print('-----------------------------------------')
print('Classification Report')
print(classification_report(y_test, multi_clf.predict(X_test), digits=4))

Accuracy score 0.8534090909090909
-----------------------------------------
Confusion Matrix
[[3079  921]
 [ 240 3680]]
-----------------------------------------
Classification Report
              precision    recall  f1-score   support

           0     0.9277    0.7698    0.8414      4000
           1     0.7998    0.9388    0.8637      3920

    accuracy                         0.8534      7920
   macro avg     0.8638    0.8543    0.8526      7920
weighted avg     0.8644    0.8534    0.8524      7920



In [30]:
import pickle
pickle.dump(vectorizer, open("vector.pickle", "wb"))
pickle.dump(multi_clf, open("model.pickle", "wb"))

In [32]:
from xgboost.sklearn import XGBClassifier

In [33]:
model = XGBClassifier(max_depth=4, learning_rate=0.2, n_estimators=300)
model.fit(X_train,y_train)

/home/vladimir/anaconda3/envs/mtc2/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:47:47] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
print(classification_report(y_test, model.predict(X_test), digits=4))


              precision    recall  f1-score   support

           0     0.9865    0.9688    0.9775      4000
           1     0.9687    0.9865    0.9775      3920

    accuracy                         0.9775      7920
   macro avg     0.9776    0.9776    0.9775      7920
weighted avg     0.9777    0.9775    0.9775      7920



In [36]:
pickle.dump(model, open("model.pickle", "wb"))

In [51]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors

data['dop'] = 1
group_names = ['user', 'tech']
group_colors = ['#F56F45', '#FA7E7E']
well = data['dop'].values
operations_numb = data['target'].values

plt.figure(figsize=(40,20))

for i, wn in enumerate(np.unique(well)):
    ax = plt.subplot(3, 4, i+1)
    hist = np.histogram(operations_numb[well == wn], bins=np.arange(3)+.2)
    plt.bar(np.arange(len(hist[0])), hist[0], color=group_colors, align='center')
    ax.set_title('Номер площадки: ' + str(wn))
    ax.set_xticks(np.arange(len(hist[0])))
    ax.set_xticklabels(group_names, rotation=90)

In [53]:
len(y_test)

7920

In [55]:
print(classification_report(y_test, np.random.randint(2, size=len(y_test)), digits=4))


              precision    recall  f1-score   support

           0     0.5024    0.4990    0.5007      4000
           1     0.4923    0.4957    0.4940      3920

    accuracy                         0.4973      7920
   macro avg     0.4973    0.4973    0.4973      7920
weighted avg     0.4974    0.4973    0.4974      7920

